## what i need to do find the probablities i need to decide what message is spam and what message is ham.
## the probabilities i need are
### P(ham)
### P(spam)
### p(word | spam)
### p(word | ham)
### p(word)



# 1 read the data

In [333]:
from collections import defaultdict
import csv
d = {}
with open('dataset.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    d = reader
# print(d)
    

     


#### next i will Write a function that will read the data from the file and return the text of the text of the email by removing any punctuation

In [334]:
import string

def remove_punctuation(text):
    # Create translation table with punctuation characters mapped to None
    translation_table = str.maketrans("", "", string.punctuation)
    
    # Remove punctuation using the translation table
    text_without_punctuation = [word.translate(translation_table) for word in text]
    
    return text_without_punctuation
# print(remove_punctuation("don't"))
def get_clean_text(text):
    
    
    commons = set(['the', 'is', 'are', 'and', 'that', 'do', 'have'])
    text = text.split()
    text = remove_punctuation(text)
    # print(text)
    clean_text = []
    for word in text:
        if word in commons: 
            continue
        if word.endswith('s'):
            clean_text.append(word[:-1])
        else:
            clean_text.append(word)
    return clean_text

            
      


## first decide how much of the data i want to train on

# lets decide to learn from the first 5000 data points

In [335]:
learning_size = 5000

#### next i will write a function that builds a dictionary of type and clean text for learning date of the dataset

In [349]:
from collections import defaultdict
import csv
def learning_data_builder():
    with open('dataset.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
    
        learning_data = []
        i  = 0
        for row in reader:
            # print(type)
            if i == learning_size:
                break
            # print(row)
            learning_data.append([row[0], row[1]])
            i += 1
        return learning_data
               
# learning_data_builder()

  

In [351]:
learning_data = learning_data_builder()


['type', 'text']
['ham', 'Hope you are having a good week. Just checking in']
['ham', 'K..give back my thanks.']
['ham', 'Am also doing in cbe only. But have to pay.']
['spam', 'complimentary 4 STAR Ibiza Holiday or �10,000 cash needs your URGENT collection. 09066364349 NOW from Landline not to lose out! Box434SK38WP150PPM18+']
['spam', 'okmail: Dear Dave this is your final notice to collect your 4* Tenerife Holiday or #5000 CASH award! Call 09061743806 from landline. TCs SAE Box326 CW25WX 150ppm']
['ham', 'Aiya we discuss later lar... Pick u up at 4 is it?']
['ham', 'Are you this much buzy']
['ham', 'Please ask mummy to call father']
['spam', 'Marvel Mobile Play the official Ultimate Spider-man game (�4.50) on ur mobile right now. Text SPIDER to 83338 for the game & we ll send u a FREE 8Ball wallpaper']
['ham', "fyi I'm at usf now, swing by the room whenever"]
['ham', 'Sure thing big man. i have hockey elections at 6, shouldn�����t go on longer than an hour though']
['ham', 'I anythin

# lets count the number of hams and spams

In [338]:
def spam_counter(learning_data):
    spams = 0
    hams = 0
    for type, value in learning_data:
        if type == 'spam':
            spams += 1
        else:
            hams += 1
    return [spams, hams]

        

In [339]:
no_spams, no_hams = spam_counter(learning_data)

# lets start calculating the probabilities we want

In [340]:
from collections import Counter


def probability_counter(learning_data):
    hams = []
    spams = []
    for type, text in learning_data:
        if type == "ham":
            hams.extend(text)
        else:
            spams.extend(text)
    ham_words = Counter(hams)
    spam_words = Counter(spams)
    word_given_ham = defaultdict(float)
    word_given_spam = defaultdict(float)

    for word in ham_words:
        word_given_ham[word] = ham_words[word] / len(hams)
    for word in spam_words:
        word_given_spam[word] = spam_words[word] / len(spams)
    return word_given_spam, word_given_ham





In [341]:
word_given_spam, word_given_ham = probability_counter(learning_data)
probability_spam, probability_ham = no_hams/5000, no_spams/5000
print(probability_spam, probability_ham)


0.8654 0.1346


# lets write the tester for our model

In [342]:

import math


def tester(text):
    text = text.split()
    probability_ham_given_featuters = probability_ham
    probability_spam_given_featuters = probability_spam
    
    for word in text:
        if word not in word_given_ham:
            probability_ham_given_featuters += math.log(1/ (no_hams + 10))
        else:
            probability_ham_given_featuters *= math.log(word_given_ham[word])
        if word not in word_given_spam:
            probability_spam_given_featuters += math.log(1/ (no_spams + 10))
        else:
            probability_spam_given_featuters += math.log(word_given_spam[word])
    if probability_spam_given_featuters  > probability_ham_given_featuters:
        return 'spam'
    else:
        return 'ham'




        


## now that we have trained the model, lets test it on the rest of the dataset

In [346]:
def test_runner():
    with open('dataset.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        i = 0
        correct = 0
        for row in reader:
            print(row)
            if i == learning_size:
                break
            if row[0] == tester(row[1]):
                correct += 1
            i += 1
    print(f'from {learning_size} {(correct / learning_size) * 100}% passed')
print(learning_data)
test_runner()


    


[['type', 'text'], ['ham', 'Hope you are having a good week. Just checking in'], ['ham', 'K..give back my thanks.'], ['ham', 'Am also doing in cbe only. But have to pay.'], ['spam', 'complimentary 4 STAR Ibiza Holiday or �10,000 cash needs your URGENT collection. 09066364349 NOW from Landline not to lose out! Box434SK38WP150PPM18+'], ['spam', 'okmail: Dear Dave this is your final notice to collect your 4* Tenerife Holiday or #5000 CASH award! Call 09061743806 from landline. TCs SAE Box326 CW25WX 150ppm'], ['ham', 'Aiya we discuss later lar... Pick u up at 4 is it?'], ['ham', 'Are you this much buzy'], ['ham', 'Please ask mummy to call father'], ['spam', 'Marvel Mobile Play the official Ultimate Spider-man game (�4.50) on ur mobile right now. Text SPIDER to 83338 for the game & we ll send u a FREE 8Ball wallpaper'], ['ham', "fyi I'm at usf now, swing by the room whenever"], ['ham', 'Sure thing big man. i have hockey elections at 6, shouldn�����t go on longer than an hour though'], ['ham